<a href="https://colab.research.google.com/github/stevenhirsch/hockey-xG-quality-vs-quantity-of-shots/blob/main/Hockey_xG_Quantity_versus_Quality_of_Shots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
np.random.seed(0)

In [2]:
potential_xg = np.linspace(1,7,1000)

In [3]:
def create_team_chances(xg, shots_a, shots_b):
    team_a = xg/shots_a
    team_b = xg/shots_b
    return team_a, team_b

In [4]:
def sim_game(chances_a, chances_b, shots_a, shots_b):
    goals_a = 0
    shots_a = shots_a
    goals_b = 0
    shots_b = shots_b
    for shot in range(shots_a):
        event = np.random.rand()
        if chances_a > event:
            goals_a += 1
    
    for shot in range(shots_b):
        event = np.random.rand()
        if chances_b > event:
            goals_b += 1
    return goals_a, goals_b

In [5]:
def get_result(goals_a, goals_b):
    win_a = 0
    win_b = 0
    if goals_a == goals_b:
        tiebreak = np.random.rand()
        if tiebreak >= 0.5:
            win_a += 2
            win_b += 1
        else:
            win_b += 2
            win_a += 1
    if goals_a > goals_b:
        win_a +=2
    if goals_b > goals_a:
        win_b += 2
    
    return win_a, win_b

In [6]:
def sim_xPTS(xg, shots_a, shots_b, num_sims, ppw = 2.0):
    pts_a = 0
    pts_b = 0
    
    chances_a, chances_b = create_team_chances(2, shots_a, shots_b)

    for game in range(num_sims):
        goals_a, goals_b = sim_game(chances_a, chances_b, shots_a=shots_a, shots_b= shots_b)
        temp_pts_a, temp_pts_b = get_result(goals_a, goals_b)
        pts_a += temp_pts_a
        pts_b += temp_pts_b
    
    xpts_a = pts_a / num_sims
    xpts_b = pts_b / num_sims
    
    return xpts_a, xpts_b

In [7]:
%%time
xPTS_a = np.array([])
xPTS_b = np.array([])
NUM_SIMS = 10**5
for i in potential_xg:
    xg_result = sim_xPTS(xg = i, shots_a = 20, shots_b =  30, num_sims = NUM_SIMS)
    xPTS_a = np.append(xPTS_a, xg_result[0])
    xPTS_b = np.append(xPTS_b, xg_result[1])

# Output for 10**5 sims on 2021 Macbook Pro (M1 Pro, 10 core CPU)
#CPU times: user 17min 24s, sys: 4.72 s, total: 17min 29s
#Wall time: 17min 30s

# Output for 10**5 sims on Google Colab
#CPU times: user 35min 14s, sys: 3.53 s, total: 35min 18s
#Wall time: 35min 21s

CPU times: user 36min 15s, sys: 4.5 s, total: 36min 20s
Wall time: 36min 26s


In [8]:
mean_xPTS_a = np.mean(xPTS_a)
sd_xPTS_a = np.std(xPTS_a)
mean_xPTS_b = np.mean(xPTS_b)
sd_xPTS_b = np.std(xPTS_b)

In [9]:
mean_xPTS_a, sd_xPTS_a

(1.10966012, 0.002939435691693221)

In [10]:
mean_xPTS_b, sd_xPTS_b

(1.10343595, 0.0029241865018326046)

In [11]:
(mean_xPTS_b - mean_xPTS_a) / np.sqrt((sd_xPTS_a**2 + sd_xPTS_b**2)/2)

-2.1229706644034927